In [10]:
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as pl
from scipy.interpolate import InterpolatedUnivariateSpline

molecule = 'Kr'

#Load in all the important premade files, and Adjust any tunables
#geometry = np.loadtxt('%s_geometry.txt' % molecule) #Should be a 3 columned XYZ Array.  Column 0 = X, Column 1 = Y, Column 2 = Z
geometry = np.array([0, 0, 0])
#geometry = geometry[np.newaxis,:]
Scattering_Factor_Coefficients = np.load('Scattering_Factors.npy', allow_pickle=True) #For calculating atomic scattering factors for elastic scattering
Compton_Factors = np.load('Compton_Factors.npy') #For calculating the inelastic scattering
atomic_numbers = np.array([36]) #These should be the atomic numbers corresponding to the rows in the geometry file.  This indicates row 0 is the XYZ coordinates for S, rows 1-6 are the XYZ coordinates for the F atoms
natoms = atomic_numbers.size 
q_bins = 1000
phi_bins = 720

#Define Normalization and other variables
q = np.linspace(0, 10, q_bins, endpoint=True)
q = np.reshape(np.repeat(q, phi_bins), (q_bins, phi_bins))
phi = np.linspace(0, 2*np.pi, phi_bins, endpoint=False)
phi = np.transpose(np.reshape(np.repeat(phi, q_bins), (phi_bins, q_bins)))

    
#This loop calculates the atomic scattering factors from the coefficients, and calculates the inelastic scattering contribution.  The atomic scattering factors will be used when the elastic scattering is calculated later.
q_inelastic = np.array([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1., 1.5, 2., 8., 15.]) #These go with the Compton factors array- don't change them unless Compton Array changes.
q_inelastic *= 4*np.pi
inelastic_scattering = np.zeros_like(q_inelastic)
scattering_factors = np.zeros((natoms, q_bins, phi_bins))
for i in range(0, natoms):
    atom = atomic_numbers[i] - 1
    factor_coefficient = Scattering_Factor_Coefficients[atom,:]
    inelastic_scattering += Compton_Factors[atom,:]
    atomic_factor = factor_coefficient[0]*np.exp(-factor_coefficient[4]*(q/(4*np.pi))**2) + factor_coefficient[1]*np.exp(-factor_coefficient[5]*(q/(4*np.pi))**2) + factor_coefficient[2]*np.exp(-factor_coefficient[6]*(q/(4*np.pi))**2) + factor_coefficient[3]*np.exp(-factor_coefficient[7]*(q/(4*np.pi))**2) + factor_coefficient[8]
    scattering_factors[i,:,:] = atomic_factor
    
scat_spline = InterpolatedUnivariateSpline(q_inelastic, inelastic_scattering)
Inelastic_Jungfrau_Shaped = scat_spline(q) 

#This part of the script sets up the plotting
#The saveplot function shows what a detector image may look like, without polarization effects accounted for! In real detector images, the X-ray polarization causes anisotropy in the image.
cmap = 'jet'
def saveplot(data,mmin,mmax,name): 
        ax1 = pl.subplots()
        ax1 = pl.subplot(111, polar=True)
        ax1.set_yticklabels([])
        image = ax1.pcolormesh(phi, q, data, cmap=cmap)
        image.set_clim(mmin,mmax)
        pl.colorbar(image)
        pl.savefig('{}.png'.format(name))
        pl.clf()
def saveplot_1D(data1,name):
        pl.figure()
        pl.plot(data1[:,0], data1[:,1])
        pl.savefig('{}.png'.format(name))

#Defining every r vector between 2 atoms- should be N**2 total
r = np.zeros((natoms, 3, natoms))
#This loop calculates the elastic scattering
molecular_contribution = np.zeros((q_bins, phi_bins))
atomic_contribution = np.sum(scattering_factors**2, axis = 0)
debye_image = atomic_contribution
for l in range(0,natoms):
    r[:,:,l] = geometry - geometry[l,:]
for i in range(0,natoms):
    for j in range(i+1, natoms):
        a = scattering_factors[i,:,:]
        b = scattering_factors[j,:,:]
        molecular_contribution = a*b*2*np.sinc(q*np.sqrt(r[j,0,i]**2 + r[j,1,i]**2 + r[j,2,i]**2)/np.pi)
        debye_image += molecular_contribution



#The Debye result has been produced- the next session plots and saves the result in various ways.
#saveplot(debye_image, 0,4900, 'NMM_test_w_inelastic')  #Save the Debye result as an image
#The next lines save the result as an array
Debye_Result = np.zeros((q[:,0].size,2))
Debye_Result[:,0] = q[:,0]
Debye_Result[:,1] = debye_image[:,0]
#saveplot_1D(Debye_Result, 'Debye_example_w_inelastic') #Saves an image
np.save('%s_Debye_Total.npy' % molecule, Debye_Result) #Saves as a numpy array to be loaded in and used in other scripts
#np.save('%s_Debye_Elastic.npy' % molecule, Debye_Result)


1


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

## CHD Percent Difference Calculation

In [2]:
from scipy.io import loadmat
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from scipy.interpolate import InterpolatedUnivariateSpline

CHD = loadmat('CHD_cas66_6311++g**.mat')
Hexatriene = np.load('Hexatriene_Debye_Total.npy')

q_CHD = np.array(CHD['q'])
I_CHD = np.array(CHD['I'])
q = Hexatriene[:,0]
I = Hexatriene[:,1]

CHD_Numpy = np.zeros((q_CHD.size, 2))
CHD_Numpy[:,0] = q_CHD[:,0]
CHD_Numpy[:,1] = I_CHD[:,0]
np.save('CHD_Debye_Total.npy', CHD_Numpy)

CHD_spline = InterpolatedUnivariateSpline(q_CHD, I_CHD)
CHD_HT_Shaped = CHD_spline(q) 

per_diff = 100*(I - CHD_HT_Shaped)/CHD_HT_Shaped
per_diff_result = np.zeros((I.size,2))
per_diff_result[:,0] = q
per_diff_result[:,1] = per_diff
plt.figure()
plt.plot(q,per_diff)
plt.savefig('perdiff.png')
#np.save('CHD_HT_Percent_Difference.npy',per_diff_result)

## This cell computes and plots atomic scattering factors

In [3]:
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from scipy.interpolate import InterpolatedUnivariateSpline
import seaborn as sns
font = {'fontname':'cmr10'}

Atomic_Number = np.array([6])

Scattering_Factor_Coefficients = np.load('Scattering_Factors.npy', allow_pickle=True) #For calculating atomic scattering factors for elastic scattering
q_bins = 1000

#Define Normalization and other variables
q = np.linspace(0, 10, q_bins, endpoint=True)

atom = Atomic_Number[0] - 1
factor_coefficient = Scattering_Factor_Coefficients[atom,:]
atomic_factor = factor_coefficient[0]*np.exp(-factor_coefficient[4]*(q/(4*np.pi))**2) + factor_coefficient[1]*np.exp(-factor_coefficient[5]*(q/(4*np.pi))**2) + factor_coefficient[2]*np.exp(-factor_coefficient[6]*(q/(4*np.pi))**2) + factor_coefficient[3]*np.exp(-factor_coefficient[7]*(q/(4*np.pi))**2) + factor_coefficient[8]

plt.figure()
plt.xlabel('q', **font)
plt.ylabel('I', **font)
plt.title('Atomic Scattering Factor Carbon', **font)
plt.plot(q, atomic_factor, color=sns.xkcd_rgb["cobalt"])
plt.xlim(0,10)
#plt.show()
plt.savefig('Carbon_Factor.png', dpi = 150., bbox_inches='tight')


